In [ ]:
import os
import glob
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import matplotlib.pyplot as plt
%matplotlib inline

## Mosaic tiles

In [ ]:
#Set directories
dirpath = r"path\\"
out_fp = r"output_path\filename.tif"

count = 0
for path in os.listdir(dirpath):
    if os.path.isfile(os.path.join(dirpath, path)):
        count += 1
print("Images to mosaic = " + str(count))

#Empty list for the datafiles that will be part of the mosaic
src_files_to_mosaic = []

#Open files in read only mode with rasterio and add those files to the source file list
for filename in glob.iglob(f'{dirpath}/*'):
    src = rasterio.open(filename)
    src_files_to_mosaic.append(src)

#merge function returns a single mosaic array and the transformation info
mosaic, out_trans = merge(src_files_to_mosaic)
show(mosaic)

# Copy the metadata
out_meta = src.meta.copy()

# Update the metadata
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 "crs":src.crs
                 }
                )

with rasterio.open(out_fp, "w", **out_meta) as dest:
    dest.write(mosaic)

print("done")